In [1]:
import numpy as np
import pandas as pd

from data import train, validation
from data import X, y, numerical, categorical

In [2]:
train.head()

,attribute2,clickVolume,avgOriginalUnitPrice,avgFinalUnitPrice,ma14SalesVolume,meanAge,gender,meanEducation,maritalStatus,plus,...,brandID_33,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,attribute1_2,attribute1_3,attribute1_4
1404,4.850075,3.082306,4.281904,3.791205,6.480146,7.813408,1.380382,11.580054,2.201123,1.650476,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2184,2.910045,0.094506,1.709888,1.637009,0.059201,7.600842,0.000000,10.896544,1.410243,1.856786,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
796,4.850075,0.217541,0.994800,1.103595,0.193577,7.016162,0.849466,11.865126,0.987170,0.866500,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
25,3.880060,0.327738,1.566200,1.306937,0.509315,6.876952,0.000000,10.805739,1.645284,1.624688,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1745,4.850075,0.389791,2.506703,2.351882,0.265934,8.667627,1.162427,11.653248,3.117380,1.476989,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# Sklearn regression pipeline

## Stage 1: Feature Engineering + Feature Selection
We first use a grid search to consider:
- Power transformations on the response variable
- Adding polynomial terms for predictors 
- Adding interaction terms for predictors
Within each parameter combination, models additionally undergo a backwards variable selection procedure using validation MSE as a metric. 

### Setup for grid search

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

pipeline = dict()
param_grid = dict()

### Feature engineering (Preprocessing)

We create a preprocessing pipeline (`preprocessor`) to later embed into the overall model pipeline. The preprocessor is kept seperate so that it can also be transferred to other models.

We consider optionally adding polynomial and interaction terms of up to degree 3 (`PolynomialFeatures`), one of the feature selection steps outlined above, as well as a power transformation (`PowerTransformer`) to normalize the response.

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import PolynomialFeatures

pipeline['preprocessing'] = Pipeline([
    ('categorical', 'passthrough'),
    ('numerical', ColumnTransformer([
        ('poly', PolynomialFeatures(), numerical),
        ('drop', VarianceThreshold(), numerical),
    ])),
])

param_grid['preprocessing'] = {
    'numerical__poly': [
        PolynomialFeatures(degree=3, include_bias=False),
        PolynomialFeatures(degree=2, include_bias=False),
        'passthrough'
    ]
}

### Feature selection

1. RFE (Recursive feature Elimination) removes variables with low coefficients. 
  - O\[k\] at each step, k is number of cross validation folds. 
  - May have trouble finding global optimum. 
  - *Guaranteed* local optimum.

2. Sequential feature selection removes variables by test MSE.
  - O\[kn\] at each step, k is number of cross validation folds, n is number of features.
  - Intuitively feels more likely to find global optimum.
  - However, cannot even guarantee a local optimum without some manual searching.


In [19]:
from sklearn.feature_selection import RFECV, SequentialFeatureSelector
from sklearn.linear_model import LinearRegression

pipeline['dim_reduction'] = Pipeline([
    ('method', 'passthrough'),
])

param_grid['dim_reduction'] = {
    'method': [
        SequentialFeatureSelector(
            estimator = LinearRegression(),
            direction = 'backward',
            scoring = 'neg_mean_squared_error',
            n_jobs=-1,
        ),
        RFECV(
            estimator = LinearRegression(),
            scoring = 'neg_mean_squared_error',
            n_jobs=-1,
        ),
        'passthrough',
    ],
}


### Standard OLS model

Having previously noted the non-normal distribution of the response, we consider a standard OLS model with an optional power transformation to normalize the response. 

Regularization will be considered in the next stage once optimal input parameters have been found.

In [20]:
from sklearn.compose import TransformedTargetRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PowerTransformer

pipeline['model_fitting'] = Pipeline([
    ('method', 'passthrough'), # Set in param grid
])

param_grid['model_fitting'] = {
    'method': [
        TransformedTargetRegressor(
            regressor = LinearRegression(),
            transformer = PowerTransformer(),
        ),
        LinearRegression(),
    ]
}

### Final Pipeline

In [40]:
from sklearn.model_selection import GridSearchCV

search = GridSearchCV(
    estimator = Pipeline([
        (name, pipe) 
        for name, pipe in pipeline.items()
    ]),
    param_grid = {
        f'{name}__{param}': value 
        for name, subgrid in param_grid.items()
        for param, value in subgrid.items()
    },
    scoring = ['r2', 'neg_mean_squared_error'],
    refit = 'neg_mean_squared_error',
    return_train_score = True,
    verbose = 10,
)

In [41]:
print(80*'=')
print('Pipeline Structure')
print(80*'-')
print(search.estimator)


Pipeline Structure
--------------------------------------------------------------------------------
Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('categorical', 'passthrough'),
                                 ('numerical',
                                  ColumnTransformer(transformers=[('poly',
                                                                   PolynomialFeatures(),
                                                                   ['attribute2',
                                                                    'clickVolume',
                                                                    'avgOriginalUnitPrice',
                                                                    'avgFinalUnitPrice',
                                                                    'ma14SalesVolume',
                                                                    'meanAge',
                                                                    'gender',


In [42]:
print(80*'=')
print('Parameter Space')
print(80*'-')
for param, values in search.param_grid.items():
    print(param)
    for value in values:
        print('-', value)
    print()

Parameter Space
--------------------------------------------------------------------------------
preprocessing__numerical__poly
- PolynomialFeatures(degree=3, include_bias=False)
- PolynomialFeatures(include_bias=False)
- passthrough

dim_reduction__method
- RFECV(estimator=LinearRegression(), n_jobs=-1, scoring='neg_mean_squared_error')
- SequentialFeatureSelector(direction='backward', estimator=LinearRegression(),
                          n_jobs=-1, scoring='neg_mean_squared_error')
- passthrough

model_fitting__method
- TransformedTargetRegressor(regressor=LinearRegression(),
                           transformer=PowerTransformer())
- LinearRegression()



### Grid Search

In [ ]:
import pickle 

search.fit(train[X], train[y])

# Save results
with open('linreg.p', 'wb') as f:
    pickle.dump(search, f)

### Stage 2: Model regularization
Model inputs are now fixed and we consider usign a mix of L1 and L2 regularization (ElasticNet) to finely control the strength of the penalty for large coefficients. The best set of regularization parameters are obtained through a grid search.